# Clustering Analysis Mannheim

## 1. Housekeeping 

In [1]:
import numpy as np
import pandas as pd
import requests
import json 

from pandas.io.json import json_normalize


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans
from sklearn import preprocessing

import folium 

print('Libraries imported.')

Libraries imported.


## 2. Get Initial Data

##### We use open source data from the municipality of Mannheim to form our initial dataset. The dataset comprises the different districts with the latest federal election data and the demographic structures (captured by age distribution and population growth).

##### Population growth
https://mannheim.opendatasoft.com/explore/dataset/bevolkerungsbestand-in-mannheim-2013-2018/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B

##### Age structure
https://mannheim.opendatasoft.com/explore/dataset/altersstruktur-in-mannheim-2013-2018/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B

##### Federal election results 2017
https://www.mannheim.de/sites/default/files/2017-09/BTW2017_amtliche_Endergebnisse_ST.xls (requires extensive data wrangling - therefore file is uploaded and not taken from the web)

#### 2.1 Preperation of popoulation growth data

In [2]:
df_popgrowth = pd.read_csv('https://mannheim.opendatasoft.com/explore/dataset/bevolkerungsbestand-in-mannheim-2013-2018/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B', delimiter=';')
df_popgrowth.head()

,ID,Jahr,Gemeindeteilschlüssel,Gemeindeteilebene,Gemeindeteilname,Bevölkerung am Ort der Hauptwohnung insgesamt,Bevölkerung am Ort der Hauptwohnung weiblich,Bevölkerung am Ort der Hauptwohnung ohne Migrationshintergrund,Bevölkerung am Ort der Hauptwohnung Migrationshintergrund,Bevölkerung am Ort der Hauptwohnung Ausländer
0,3,2018,20,Stadtteil,Neckarstadt-West,21451,9839,6665,14786,10974
1,39,2017,92,Stadtteil,Neuhermsheim,4556,2343,2662,1894,684
2,114,2014,92,Stadtteil,Neuhermsheim,4614,2380,2712,1902,718
3,123,2014,162,Stadtteil,Niederfeld,7278,3835,5254,2024,971
4,122,2014,161,Stadtteil,Neckarau,15539,7796,9744,5795,3422


In [3]:
# drop unecessary rows
df_popgrowth.drop(columns=['ID','Bevölkerung am Ort der Hauptwohnung weiblich','Bevölkerung am Ort der Hauptwohnung ohne Migrationshintergrund','Bevölkerung am Ort der Hauptwohnung Migrationshintergrund','Gemeindeteilebene','Bevölkerung am Ort der Hauptwohnung Ausländer'], inplace=True)


In [5]:
# name cloumns in english
df_popgrowth.rename(columns={'Jahr':'Year', 
                             'Gemeindeteilschlüssel':'District_ID',
                             'Gemeindeteilname':'District_Name',
                             'Bevölkerung am Ort der Hauptwohnung insgesamt': 'Population'},inplace=True)

In [6]:
df_popgrowth.sort_values(by=['District_ID','Year'],inplace=True)
df_popgrowth.reset_index(drop=True,inplace=True)
# total population of Mannheim with district ID of 0
df_popgrowth.head(10)

,Year,District_ID,District_Name,Population
0,2013,0,Mannheim,307936
1,2014,0,Mannheim,311470
2,2015,0,Mannheim,317744
3,2016,0,Mannheim,316265
4,2017,0,Mannheim,318910
5,2018,0,Mannheim,320080
6,2013,10,Innenstadt/Jungbusch,29698
7,2014,10,Innenstadt/Jungbusch,30180
8,2015,10,Innenstadt/Jungbusch,30408
9,2016,10,Innenstadt/Jungbusch,29851


In [7]:
# transform to pivto table to calculate CAGR of Population
df_piv = df_popgrowth.pivot_table('Population', ['District_ID','District_Name'], 'Year')
# CAGR calculation + addistional column for current population(2018)
df_piv['Population_2018'] = df_piv[[2018]]
df_piv['CAGR'] = df_piv.iloc[:, -1].div(df_piv.iloc[:, 0]).pow(1./(len(df_piv.columns) - 1)).sub(1)


In [8]:
# transform back to normal table to ensure caompatibility with other tables
df_piv.columns.name = None              
df_piv = df_piv.reset_index()


In [9]:
# filter rows
df_popgr = df_piv.filter(['District_ID','District_Name','Population_2018','CAGR'], axis=1)

#### 2.2 Preperation of Age distribution data

In [11]:
# load csv , drop columns, rename columns, sort columns
df_age_dis = pd.read_csv('https://mannheim.opendatasoft.com/explore/dataset/altersstruktur-in-mannheim-2013-2018/download/?format=csv&timezone=Europe/Berlin&lang=en&use_labels_for_header=true&csv_separator=%3B', delimiter=';')
df_age_dis.drop(columns=['ID','Gemeindeteilebene'], inplace=True)
df_age_dis.rename(columns={'Jahr':'Year', 
                             'Gemeindeteilschlüssel':'District_ID',
                             'Gemeindeteilname':'District_Name', 'Alter':'Age',
                             'Bevölkerung am Ort der Hauptwohnung': 'Population'},inplace=True)

In [12]:
# only keep the most current entries (2018)
df_age_dis = df_age_dis[df_age_dis['Year'] == 2018]
# take care of 90 + and 85-90 and convert to integer - later we will sum all people over 84
df_age_dis.loc[df_age_dis['Age'] == '90+', 'Age'] = 90
df_age_dis.loc[df_age_dis['Age'] == '85-89', 'Age'] = 85
df_age_dis['Age'] = df_age_dis['Age'].astype(int)
df_age_dis.sort_values(by=['District_ID','Age'],inplace=True)
df_age_dis.reset_index(drop=True,inplace=True)


In [13]:
# create a pivot table
df_piv = df_age_dis.pivot_table('Population', ['District_ID','District_Name','Year'], 'Age')

In [14]:
#group results
g1 = list(range(0,13))  # 0-12
g2 = list(range(13,18)) # 13-17
g3 = list(range(18,35)) # 18-34
g4 = list(range(35,60)) # 35-59
g5 = list(range(60,85)) # 60-84
g6 = [85,90]            # 85-90

df_piv['Group 0-10'] = df_piv.loc[:,g1].sum(axis=1)
df_piv['Group 13-17'] = df_piv.loc[:,g2].sum(axis=1)
df_piv['Group 18-34'] = df_piv.loc[:,g3].sum(axis=1)
df_piv['Group 35-59'] = df_piv.loc[:,g4].sum(axis=1)
df_piv['Group 60-84'] = df_piv.loc[:,g5].sum(axis=1)
df_piv['Group 85-90'] = df_piv.loc[:,g6].sum(axis=1)


In [15]:
# add percentage
total = list(range(0,85))
total.append(85)
total.append(90)
df_piv['Total sum'] = df_piv.loc[:,total].sum(axis=1)
df_piv['Group 0-10 Perc'] = df_piv['Group 0-10']/df_piv['Total sum']
df_piv['Group 13-17 Perc'] = df_piv['Group 13-17']/df_piv['Total sum']
df_piv['Group 18-34 Perc'] = df_piv['Group 18-34']/df_piv['Total sum']
df_piv['Group 35-59 Perc'] = df_piv['Group 35-59']/df_piv['Total sum']
df_piv['Group 60-84 Perc'] = df_piv['Group 60-84']/df_piv['Total sum']
df_piv['Group 85-90 Perc'] = df_piv['Group 85-90']/df_piv['Total sum']

In [16]:
# transform back to normal table to ensure caompatibility with other tables
df_piv.columns.name = None              
df_piv = df_piv.reset_index()

In [17]:
# filter rows
df_age_dis = df_piv.filter(['District_ID','District_Name','Group 0-10 Perc','Group 13-17 Perc','Group 18-34 Perc','Group 35-59 Perc','Group 60-84 Perc','Group 85-90 Perc'], axis=1)
df_age_dis.head()

,District_ID,District_Name,Group 0-10 Perc,Group 13-17 Perc,Group 18-34 Perc,Group 35-59 Perc,Group 60-84 Perc,Group 85-90 Perc
0,0,Mannheim,0.108976,0.039778,0.259788,0.348610,0.217293,0.025556
1,10,Innenstadt/Jungbusch,0.076985,0.027722,0.425945,0.312254,0.142958,0.014135
2,20,Neckarstadt-West,0.121346,0.042935,0.330148,0.350333,0.145355,0.009883
3,30,Neckarstadt-Ost,0.110936,0.036880,0.284141,0.340410,0.204116,0.023518
4,41,Oststadt,0.097534,0.025613,0.264848,0.353342,0.228372,0.030291


#### 2.3 Preperation of Federal Election Data

In [18]:
df_elec = pd.read_csv('Bundestagswahl2017_data.csv',delimiter=';',decimal=',')
# drop NaNs
df_elec.dropna(inplace=True)
# Transform to integer
df_elec['District_ID'] = df_elec[['District_ID']].astype(int)
# filter necessary rows
df_elec = df_elec.filter(['District_ID','CDU','SPD','GRÜNE','FDP','AfD','DIE LINKE'])
# group by District ID because there are multiple lines with same district id 
df_elec = df_elec.groupby('District_ID').mean()
# transform to normal table
df_elec.columns.name = None               
df_elec = df_elec.reset_index()             

df_elec.head()

,District_ID,CDU,SPD,GRÜNE,FDP,AfD,DIE LINKE
0,0,27.147692,21.236241,13.174471,11.245560,12.828346,9.100846
1,10,20.781361,17.158023,19.920327,12.861429,7.188140,14.888199
2,20,15.930265,18.646705,18.579415,7.518993,10.592156,18.187347
3,30,21.962129,22.458502,14.825784,8.274282,12.879994,12.896270
4,41,26.280018,16.175804,17.874943,17.263253,8.699592,10.172179


#### 2.4 Join tables

In [19]:
#full outer join 
df_merged = pd.merge(left=df_popgr, right=df_age_dis, how='outer', left_on='District_ID', right_on='District_ID')
df_merged = pd.merge(left=df_merged, right=df_elec, how='outer', left_on='District_ID', right_on='District_ID')
df_merged.drop(columns=['District_Name_y'], inplace=True)
df_merged.rename(columns={'District_Name_x':'District_Name'},inplace=True)
df_merged.head()

,District_ID,District_Name,Population_2018,CAGR,Group 0-10 Perc,Group 13-17 Perc,Group 18-34 Perc,Group 35-59 Perc,Group 60-84 Perc,Group 85-90 Perc,CDU,SPD,GRÜNE,FDP,AfD,DIE LINKE
0,0,Mannheim,320080,0.006467,0.108976,0.039778,0.259788,0.348610,0.217293,0.025556,27.147692,21.236241,13.174471,11.245560,12.828346,9.100846
1,10,Innenstadt/Jungbusch,31058,0.007491,0.076985,0.027722,0.425945,0.312254,0.142958,0.014135,20.781361,17.158023,19.920327,12.861429,7.188140,14.888199
2,20,Neckarstadt-West,21451,0.006745,0.121346,0.042935,0.330148,0.350333,0.145355,0.009883,15.930265,18.646705,18.579415,7.518993,10.592156,18.187347
3,30,Neckarstadt-Ost,33677,0.009440,0.110936,0.036880,0.284141,0.340410,0.204116,0.023518,21.962129,22.458502,14.825784,8.274282,12.879994,12.896270
4,41,Oststadt,12611,0.006392,0.097534,0.025613,0.264848,0.353342,0.228372,0.030291,26.280018,16.175804,17.874943,17.263253,8.699592,10.172179


## 3. Add location and venue data

#### 3.1 Location Data

##### Handselected location data for districts in Mannheim. The location was set so that the pin (lat,lon) is in the center of the populated area of the respective district. 

In [20]:
df_loc = pd.read_csv('Geodata_Mannheim_Districts.csv',delimiter=';')
df_loc.head()
df_loc['Lon']=df_loc[['Lon']].astype(float)
df_loc['Lat']=df_loc[['Lat']].astype(float)
df_loc.drop(columns={'District_Name'},inplace=True)
df_loc.head()

,District_ID,Lon,Lat
0,10,8.466033,49.487664
1,20,8.466757,49.501595
2,30,8.479452,49.496859
3,41,8.483248,49.483311
4,42,8.478261,49.478243
5,50,8.468874,49.469997
6,60,8.448417,49.544324
7,70,8.473385,49.544377
8,81,8.489008,49.523885
9,82,8.497723,49.529699


In [21]:
# join the table with the merged table
df_merged = pd.merge(left=df_merged, right=df_loc, how='outer', left_on='District_ID', right_on='District_ID')
# reorder lat and lon to the front
cols = df_merged.columns.tolist()
cols = cols[:2] + cols[-2:] + cols[3:-2]
df_merged = df_merged[cols]
# get rid of the Mannheim row as we are interetested in the distinct districts
df_merged = df_merged[df_merged['District_ID']!=0]
df_merged.head()


,District_ID,District_Name,Lon,Lat,CAGR,Group 0-10 Perc,Group 13-17 Perc,Group 18-34 Perc,Group 35-59 Perc,Group 60-84 Perc,Group 85-90 Perc,CDU,SPD,GRÜNE,FDP,AfD,DIE LINKE
1,10,Innenstadt/Jungbusch,8.466033,49.487664,0.007491,0.076985,0.027722,0.425945,0.312254,0.142958,0.014135,20.781361,17.158023,19.920327,12.861429,7.188140,14.888199
2,20,Neckarstadt-West,8.466757,49.501595,0.006745,0.121346,0.042935,0.330148,0.350333,0.145355,0.009883,15.930265,18.646705,18.579415,7.518993,10.592156,18.187347
3,30,Neckarstadt-Ost,8.479452,49.496859,0.009440,0.110936,0.036880,0.284141,0.340410,0.204116,0.023518,21.962129,22.458502,14.825784,8.274282,12.879994,12.896270
4,41,Oststadt,8.483248,49.483311,0.006392,0.097534,0.025613,0.264848,0.353342,0.228372,0.030291,26.280018,16.175804,17.874943,17.263253,8.699592,10.172179
5,42,Schwetzingerstadt,8.478261,49.478243,0.012639,0.074088,0.019788,0.416018,0.329926,0.145456,0.014724,26.857930,16.984990,19.055650,15.479874,6.770012,10.106824


#### 3.2 Intermediate Result - view the map of Mannheim

In [22]:
# create map of New York using latitude and longitude values
latitude = 49.488888
longitude = 8.469167
map_mannheim = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lon, District_Name, District_ID in zip(df_merged['Lat'], df_merged['Lon'], df_merged['District_Name'], df_merged['District_ID']):
    label = '{}, {}'.format(District_Name, District_ID)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mannheim)  
    
map_mannheim

#### 3.3 Add venue data

##### Use the foursquare application to get venue data for the different districts

In [24]:
CLIENT_ID = 'F5QDH1IKMFRGZJXLWU3SWG0YZLJ5JIJSNXMGPOPBCOE3XLN3' # Foursquare ID
CLIENT_SECRET = 'PTFITORAP33KKSSWFPTRG0QWN4DI5SC011IUTKQETLWVTUTY' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [25]:
# define function to get nearby venues
def getNearbyVenues(names, IDs, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, ID, lat, lng in zip(names, IDs, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            ID,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District_Name', 
                             'District_ID',
                  'District_Lat', 
                  'District_Lon', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return (nearby_venues)

In [26]:
LIMIT = 200
mannheim_venues = getNearbyVenues(names=df_merged['District_Name'], IDs=df_merged['District_ID'],
                                   latitudes=df_merged['Lat'],
                                   longitudes=df_merged['Lon'])

Innenstadt/Jungbusch
Neckarstadt-West
Neckarstadt-Ost
Oststadt
Schwetzingerstadt
Lindenhof
Sandhofen
Schönau
Waldhof
Gartenstadt
Luzenberg
Neuostheim
Neuhermsheim
Seckenheim
Hochstätt
Friedrichsfeld
Käfertal
Vogelstang
Wallstadt
Feudenheim
Neckarau
Niederfeld
Almenhof
Rheinau


In [27]:
mannheim_venues.head()

,District_Name,District_ID,District_Lat,District_Lon,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Innenstadt/Jungbusch,10,49.487664,8.466033,Mémoires d'Indochine,49.487279,8.464476,Vietnamese Restaurant
1,Innenstadt/Jungbusch,10,49.487664,8.466033,Café Herrdegen,49.488732,8.465936,Dessert Shop
2,Innenstadt/Jungbusch,10,49.487664,8.466033,Jamy's Burger,49.488997,8.467815,Burger Joint
3,Innenstadt/Jungbusch,10,49.487664,8.466033,Südlandhaus,49.487335,8.469226,Gourmet Shop
4,Innenstadt/Jungbusch,10,49.487664,8.466033,Café Prag,49.488841,8.463416,Café


In [28]:
mannheim_onehot = pd.get_dummies(mannheim_venues[['Venue Category']], prefix="", prefix_sep="")

# add District_Name and District_ID column back to datafbrame
mannheim_onehot['District_Name'] = mannheim_venues['District_Name']
mannheim_onehot['District_ID'] = mannheim_venues['District_ID']

# move District_Name and District_ID column to the first column
cols = mannheim_onehot.columns.tolist()
cols = cols[-2:] + cols[:-2]
mannheim_onehot = mannheim_onehot[cols]

mannheim_onehot.head()

,District_Name,District_ID,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Burger Joint,Bus Stop,Café,...,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vietnamese Restaurant,Wine Shop
0,Innenstadt/Jungbusch,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,Innenstadt/Jungbusch,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Innenstadt/Jungbusch,10,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Innenstadt/Jungbusch,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Innenstadt/Jungbusch,10,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [29]:
mannheim_grouped = mannheim_onehot.groupby('District_ID').mean().reset_index()
mannheim_grouped.head()

,District_ID,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Burger Joint,Bus Stop,Café,Chinese Restaurant,...,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vietnamese Restaurant,Wine Shop
0,10,0.000,0.016949,0.016949,0.0,0.033898,0.050847,0.0,0.135593,0.016949,...,0.000000,0.016949,0.0,0.016949,0.0,0.000,0.000000,0.067797,0.016949,0.0
1,20,0.125,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.125000,0.000000,...,0.125000,0.000000,0.0,0.000000,0.0,0.125,0.000000,0.125000,0.000000,0.0
2,30,0.000,0.000000,0.050000,0.0,0.000000,0.000000,0.0,0.200000,0.000000,...,0.150000,0.000000,0.0,0.000000,0.0,0.050,0.050000,0.000000,0.000000,0.0
3,41,0.000,0.000000,0.055556,0.0,0.000000,0.000000,0.0,0.222222,0.000000,...,0.055556,0.000000,0.0,0.055556,0.0,0.000,0.055556,0.000000,0.000000,0.0
4,42,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.150000,0.000000,...,0.100000,0.050000,0.0,0.000000,0.0,0.050,0.050000,0.000000,0.000000,0.0


#### 3.4 Join everything together

In [38]:
df_final = pd.merge(left=df_merged, right=mannheim_grouped, how='outer', left_on='District_ID', right_on='District_ID')
df_final = df_final.fillna(0)

## 4. Clustering

##### To get a profound view of the underlying dynamics 3 clusters are performed and analyzed:
##### 1. Clustering with demographical data only
##### 2. Clustering with venue data only
##### 3. Clustering with the two sets combined

In [39]:
cols = df_final.columns.tolist()
cols_cluster_1 = cols[4:(-len(cols)+17)]
cols_cluster_2 = cols[(-len(cols)+17):]
cols_cluster_3 = cols[4:]
cols_clust = (cols_cluster_1, cols_cluster_2, cols_cluster_3)
# print(cols_clust)

In [40]:
# 1. Clustering with demographical data only

kclusters = 5
l = 1
for clust in cols_clust:
    mannheim_grouped_clustering = df_final[df_final.columns.intersection(clust)]
    # normalize data
    x = mannheim_grouped_clustering.values
    standardscaler = preprocessing.StandardScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    mannheim_grouped_clustering = pd.DataFrame(x_scaled)
    
    # run k-means clustering
    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mannheim_grouped_clustering)
    # add clustering labels
    column = ('Cluster Label '+ str(l))
    df_final.insert(0,column , kmeans.labels_)
    l=l+1


In [41]:
df_cluster = df_final[['District_Name','District_ID','Lon','Lat','Cluster Label 1', 'Cluster Label 2','Cluster Label 3']]
df_cluster.head()

,District_Name,District_ID,Lon,Lat,Cluster Label 1,Cluster Label 2,Cluster Label 3
0,Innenstadt/Jungbusch,10,8.466033,49.487664,3,1,0
1,Neckarstadt-West,20,8.466757,49.501595,2,0,2
2,Neckarstadt-Ost,30,8.479452,49.496859,2,0,2
3,Oststadt,41,8.483248,49.483311,4,2,3
4,Schwetzingerstadt,42,8.478261,49.478243,3,4,4


In [51]:
# create map
clustertype = 1 #use values 1,2 or 3 corresponding to the description above
clustercolumn = 'Cluster Label '+str(clustertype)
latitude = 49.488888
longitude = 8.469167
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_cluster['Lat'], df_cluster['Lon'], df_cluster['District_Name'], df_cluster[clustercolumn]):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color='PuBu'[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters

In [49]:
df_final_grouped= df_final.groupby('Cluster Label 3').mean().reset_index()
df_final_grouped.drop(columns={'Cluster Label 2','Cluster Label 1', 'District_ID','Lon','Lat'},inplace=True)
df_final_grouped

,Cluster Label 3,CAGR,Group 0-10 Perc,Group 13-17 Perc,Group 18-34 Perc,Group 35-59 Perc,Group 60-84 Perc,Group 85-90 Perc,CDU,SPD,...,Supermarket,Sushi Restaurant,Tennis Court,Thai Restaurant,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vietnamese Restaurant,Wine Shop
0,0,0.007491,0.076985,0.027722,0.425945,0.312254,0.142958,0.014135,20.781361,17.158023,...,0.000000,0.016949,0.000000,0.016949,0.000000,0.00000,0.000000,0.067797,0.016949,0.000000
1,1,0.004537,0.110489,0.042387,0.214352,0.355553,0.245969,0.031251,30.009426,21.404732,...,0.142310,0.000000,0.017857,0.000000,0.005102,0.10334,0.000000,0.017857,0.000000,0.007937
2,2,0.006353,0.135889,0.048730,0.248307,0.353104,0.194108,0.019861,21.796322,25.153920,...,0.086905,0.000000,0.000000,0.000000,0.214286,0.02500,0.007143,0.017857,0.000000,0.000000
3,3,0.006392,0.097534,0.025613,0.264848,0.353342,0.228372,0.030291,26.280018,16.175804,...,0.055556,0.000000,0.000000,0.055556,0.000000,0.00000,0.055556,0.000000,0.000000,0.000000
4,4,0.012639,0.074088,0.019788,0.416018,0.329926,0.145456,0.014724,26.857930,16.984990,...,0.100000,0.050000,0.000000,0.000000,0.000000,0.05000,0.050000,0.000000,0.000000,0.000000


### Dataframe **df_final_grouped** contains aggregated information about the characteristics of the individual cluster that can be examined in more depth 